<a href="https://colab.research.google.com/github/ashikMostofaTonmoy/facial-emotion-recognition/blob/main/thesis_emo_det_test_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [ ]:

import os
import zipfile
import shutil
from google.colab import drive
drive.mount('/content/drive')
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
local_zip = '/content/drive/My Drive/Thesis/KDEF_and_AKDEF.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
try:
    os.makedirs('/content/Thesis_Emodet', exist_ok=True)
except OSError as error:
    print(error)
zip_ref.extractall('/content/Thesis_Emodet')
zip_ref.close()

In [ ]:
files = []
dir = []
basepath = '/content/Thesis_Emodet/KDEF_and_AKDEF/KDEF'
for entry in os.listdir(basepath):
    sub=[]
    dir.append(basepath+'/'+entry+'/')
    for file in os.listdir(basepath+'/'+entry):
        sub.append(file)
    files.append(sub)

root='/content/Thesis_Emodet/emotions/'
emotion_type=['AF','AN','DI','HA','NE','SA','SU']

for sublist in range(len(files)):
    for file in files[sublist]:
        for i in range (len(emotion_type)):
            if file[4:6] == emotion_type[i]:
                try:
                    os.makedirs(root+emotion_type[i], exist_ok=True)
                except OSError as error:
                    print(error)
                shutil.copy(dir[sublist]+file, root+emotion_type[i])
                break

In [ ]:
"""basepath2 = '/content/Thesis_Emodet/KDEF_and_AKDEF/AKDEF'
files_AKDEF = []
for entry in os.listdir(basepath2):
    files_AKDEF.append(entry)
for file in files_AKDEF:
    for i in range (len(emotion_type)):
        if file[1:3] == emotion_type[i]:
            try:
                os.makedirs(root+emotion_type[i], exist_ok=True)
            except OSError as error:
                print(error)
                #print(root+emotion_type[i]+'/'+file)
            shutil.copy(basepath2+'/'+file, root+emotion_type[i])
            break"""

"basepath2 = '/content/Thesis_Emodet/KDEF_and_AKDEF/AKDEF'\nfiles_AKDEF = []\nfor entry in os.listdir(basepath2):\n    files_AKDEF.append(entry)\nfor file in files_AKDEF:\n    for i in range (len(emotion_type)):\n        if file[1:3] == emotion_type[i]:\n            try:\n                os.makedirs(root+emotion_type[i], exist_ok=True)\n            except OSError as error:\n                print(error)\n                #print(root+emotion_type[i]+'/'+file)\n            shutil.copy(basepath2+'/'+file, root+emotion_type[i])\n            break"

In [ ]:
checkpoint_path = "/content/drive/My Drive/Thesis/test_4 cp/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)



# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

In [ ]:
# Call_back function for stopping training after a certain level


class myCallback(tf.keras.callbacks.Callback):# Your Code
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('acc')>.999):
            print("\nReached 99.9% accuracy so cancelling training!")
            self.model.stop_training = True

AccuracyLimiterCallback = myCallback()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# For data split into test & validation
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest',
    validation_split=0.15) # set validation split

batch_size=32
img_height=400
img_width=300
train_data_dir='/content/Thesis_Emodet/emotions/' # source directory for training images
train_generator = train_datagen.flow_from_directory(
    directory=train_data_dir,
    color_mode='grayscale',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # training data

validation_generator = train_datagen.flow_from_directory(
    directory=train_data_dir, # same directory as training data
    color_mode='grayscale',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # validation data




Found 4165 images belonging to 7 classes.
Found 733 images belonging to 7 classes.


In [ ]:

"""from tensorflow.keras import layers
from tensorflow.keras import Model
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /content/Thesis_Emodet/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
  
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = '/content/Thesis_Emodet/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape = (img_height , img_width, 3), 
                                include_top = False, 
                                weights = None)

pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
  layer.trainable = False
 
pre_trained_model.summary()

last_layer = pre_trained_model.get_layer('mixed8')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output
"""

"from tensorflow.keras import layers\nfrom tensorflow.keras import Model\n!wget --no-check-certificate     https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5     -O /content/Thesis_Emodet/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5\n  \nfrom tensorflow.keras.applications.inception_v3 import InceptionV3\n\nlocal_weights_file = '/content/Thesis_Emodet/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'\n\npre_trained_model = InceptionV3(input_shape = (img_height , img_width, 3), \n                                include_top = False, \n                                weights = None)\n\npre_trained_model.load_weights(local_weights_file)\n\nfor layer in pre_trained_model.layers:\n  layer.trainable = False\n \npre_trained_model.summary()\n\nlast_layer = pre_trained_model.get_layer('mixed8')\nprint('last layer output shape: ', last_layer.output_shape)\nlast_output = last_layer.output\n"

In [ ]:
"""from tensorflow.keras.optimizers import RMSprop

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.4)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense  (7, activation='softmax')(x)           

model = Model( pre_trained_model.input, x) 

model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'categorical_crossentropy', 
              metrics = ['acc'])
model.summary()
"""

"from tensorflow.keras.optimizers import RMSprop\n\n# Flatten the output layer to 1 dimension\nx = layers.Flatten()(last_output)\n# Add a fully connected layer with 1,024 hidden units and ReLU activation\nx = layers.Dense(1024, activation='relu')(x)\n# Add a dropout rate of 0.2\nx = layers.Dropout(0.4)(x)                  \n# Add a final sigmoid layer for classification\nx = layers.Dense  (7, activation='softmax')(x)           \n\nmodel = Model( pre_trained_model.input, x) \n\nmodel.compile(optimizer = RMSprop(lr=0.0001), \n              loss = 'categorical_crossentropy', \n              metrics = ['acc'])\nmodel.summary()\n"

In [ ]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(img_height , img_width, 1)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(7, activation='softmax')
])


model.summary()

model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['acc'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 398, 298, 64)      1792      
_________________________________________________________________
batch_normalization (BatchNo (None, 398, 298, 64)      256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 199, 149, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 197, 147, 64)      36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 197, 147, 64)      256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 98, 73, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 96, 71, 128)       7

In [ ]:
history = model.fit_generator(
            train_generator,
            validation_data = validation_generator,
            epochs = 700,
            callbacks=[cp_callback,AccuracyLimiterCallback],
            verbose = 1)
model.save ('/content/drive/My Drive/Thesis/test_5 cp/test_5.h5')

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 131 steps, validate for 23 steps
Epoch 1/700
 88/131 [===================>..........] - ETA: 46s - loss: 2.7241 - acc: 0.1307

In [ ]:
%matplotlib inline

import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc=history.history['acc']
val_acc=history.history['val_acc']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")
plt.figure()